In [1]:
!nvidia-smi

Sun Jul 16 08:51:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3D:00.0 Off |                    0 |
| N/A   35C    P0    42W / 300W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from pynvml import *
import torch.optim as optim


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()
    


In [3]:
print_gpu_utilization()

GPU memory occupied: 257 MB.


In [8]:
model_name = "content/t5-small-finetuned-wikisql"
from transformers import AutoTokenizer, T5ForConditionalGeneration

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [10]:
from datasets import load_dataset

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset




In [12]:
df1 = pd.read_csv('Training_set_IMDB/training_set_sample_no_target.csv')
df1 = df1.dropna()


shuffled_df = df1.sample(frac=1, random_state=42)  # Set random_state for reproducibility

train_df, eval_df = train_test_split(shuffled_df, test_size=0.1, random_state=42)


text = []
sql = []

for index, row in train_df.iterrows():
    
    text_input = "Translate to SQL: " + row['Text'],
    sql_input = row['SQL']
    text.append(text_input)
    sql.append(sql_input)
    
inputs = {"inputs": text,
        "target": sql}

train_dataset = Dataset.from_dict(inputs)

In [15]:
train_dataset['target'][0]

"SELECT r.role AS recent_role FROM actors a INNER JOIN roles r ON a.id = r.actor_id INNER JOIN movies m ON r.movie_id = m.id WHERE a.first_name = 'Pete (I)' AND a.last_name = 'Wilson' ORDER BY m.year DESC LIMIT 1"

In [16]:
text_2 = []
sql_2 = []

for index, row in eval_df.iterrows():
    
    text_input = "Translate to SQL: " + row['Text'],
    sql_input = row['SQL']
    text_2.append(text_input)
    sql_2.append(sql_input)
    
inputs_2 = {"inputs": text_2,
        "target": sql_2}
eval_dataset = Dataset.from_dict(inputs_2)

In [17]:
eval_dataset[-1]

{'inputs': ['Translate to SQL: what is actor Tom (XVI) Moore recent role?'],
 'target': "SELECT r.role AS recent_role FROM actors a INNER JOIN roles r ON a.id = r.actor_id INNER JOIN movies m ON r.movie_id = m.id WHERE a.first_name = 'Tom (XVI)' AND a.last_name = 'Moore' ORDER BY m.year DESC LIMIT 1"}

In [18]:
def map_to_lenght(x):
    x["input_len"] = len(tokenizer(x["inputs"]).input_ids)
    x["input_longer_256"] = int(x["input_len"]>256)
    x["input_longer_128"] = int(x["input_len"]>128)
    x["input_longet_64"] = int(x["input_len"]>64)
    x["output_len"] = len(tokenizer(x["target"]).input_ids)
    x["output_longet_256"] = int(x["output_len"]>256)
    x["output_longet_128"] = int(x["output_len"]>128)
    x["output_longet_64"] = int(x["output_len"]>64)
    return x

sample_size = 10000
data_stats = train_dataset.select(range(sample_size)).map(map_to_lenght, num_proc=4)

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [19]:
def compute_and_print(x):
    if len(x["input_len"])==sample_size:
        print(
            f"Input mean: {sum(x['input_len'])/sample_size} \n % of input len > 256: {sum(x['input_longer_256'])/sample_size}, \n % of input len > 128: {sum(x['input_longer_128'])/sample_size}, \n % of input len > 64: {sum(x['input_longet_64'])/sample_size}, \n Ouput mean: {sum(x['output_len'])/sample_size},\n% of output len > 256: {sum(x['output_longet_256'])/sample_size}, \n% of output len > 128: {sum(x['output_longet_128'])/sample_size}, \n% of output len > 64: {sum(x['output_longet_64'])/sample_size}")

output = data_stats.map(compute_and_print, batched=True, batch_size=-1)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Input mean: 1.0 
 % of input len > 256: 0.0, 
 % of input len > 128: 0.0, 
 % of input len > 64: 0.0, 
 Ouput mean: 106.4648,
% of output len > 256: 0.0, 
% of output len > 128: 0.2934, 
% of output len > 64: 0.7241


In [20]:
def convert_to_features(example_batch, padding = "max_length"):
    inputs = tokenizer.batch_encode_plus(example_batch["inputs"],is_split_into_words=True, max_length=64, padding="max_length", truncation=True)
    
    targets = tokenizer.batch_encode_plus(example_batch["target"], max_length=256,padding = 'max_length',truncation = True)
    if padding == "max_length":
        targets["inputs_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in target] for target in targets["input_ids"]
        ]
    
    inputs["labels"] = targets['input_ids']
    return inputs

train_data = train_dataset.map(convert_to_features, batched=True, remove_columns=train_dataset.column_names)
test_data = eval_dataset.map(convert_to_features, batched=True, remove_columns=eval_dataset.column_names)

Map:   0%|          | 0/38756 [00:00<?, ? examples/s]

Map:   0%|          | 0/4307 [00:00<?, ? examples/s]

In [22]:
test_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 4307
})

In [23]:
columns = ['input_ids', 'attention_mask', 'labels']

train_data.set_format(type='torch', columns=columns)
test_data.set_format(type='torch', columns=columns)

In [27]:
len(test_data)

4307

In [28]:
def sequence_check(list_of_seq):
    len_seq = len(list_of_seq[0])
    for seq in list_of_seq:
        if len(seq)!=len_seq:
            print(len(seq))
            return False
        
    return True

In [32]:
sequence_check(test_data['input_ids'])

True

In [33]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import os
output_dir = 'T5-fine-tuned-with-IMDB-wikisql'



In [34]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=16,
    num_train_epochs=10,
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    learning_rate=2e-4,
    weight_decay=0.1,
    do_eval=True,
    logging_strategy="epoch",
    save_strategy="epoch",
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True,
    push_to_hub=False,
    fp16=True 
)

In [35]:
from evaluate import load
rouge = load("rouge")

In [36]:
import numpy as np

def compute_metrics(pred):
    predictions, labels = pred
    
    pred_str = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    rouge_output = rouge.compute(predictions=pred_str, references=label_str)
    return {
        "rouge1": round(rouge_output['rouge1']* 100,4),
        "rouge2": round(rouge_output['rouge2']*100, 4),
        "rougeL": round(rouge_output['rougeL']*100, 4)
    }

In [37]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=-100)

In [38]:
trainer = Seq2SeqTrainer(
    model = model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset= test_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [39]:
trainer.train()

/home/toibazd/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
1,0.047800,0.006899,39.035500,36.503100,39.045800
2,0.009100,0.005094,39.035500,36.503700,39.044700
3,0.006600,0.003901,39.035500,36.503700,39.044700
4,0.005200,0.003373,39.035200,36.504000,39.045400
5,0.004200,0.003175,39.035500,36.503100,39.045400
6,0.003600,0.002929,39.035600,36.504400,39.045800
7,0.003100,0.002774,39.035600,36.504400,39.045800
8,0.002700,0.002746,39.035600,36.504400,39.045800
9,0.002500,0.002736,39.035600,36.504400,39.045800
10,0.002300,0.002705,39.035600,36.504400,39.045800


TrainOutput(global_step=24230, training_loss=0.00870360885333582, metrics={'train_runtime': 4481.7871, 'train_samples_per_second': 86.474, 'train_steps_per_second': 5.406, 'total_flos': 6556633572311040.0, 'train_loss': 0.00870360885333582, 'epoch': 10.0})

In [40]:
trainer.save_model(output_dir)

In [41]:
tokenizer.save_pretrained(output_dir)

('T5-fine-tuned-with-IMDB-wikisql/tokenizer_config.json',
 'T5-fine-tuned-with-IMDB-wikisql/special_tokens_map.json',
 'T5-fine-tuned-with-IMDB-wikisql/tokenizer.json')